<a href="https://colab.research.google.com/github/yeoun9/torchpm/blob/main/examples/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
! pip install git+https://github.com/yeoun9/torchpm.git
! pip install sympytorch

  Running command git clone --filter=blob:none --quiet https://github.com/yeoun9/torchpm.git 'C:\Users\sungwoo\AppData\Local\Temp\pip-req-build-6qk4p0t5'



  Cloning https://github.com/yeoun9/torchpm.git to c:\users\sungwoo\appdata\local\temp\pip-req-build-6qk4p0t5
  Resolved https://github.com/yeoun9/torchpm.git to commit a98641c57b7241a6e47e20b97d7c26d980ee80f4
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for torchpm: filename=torchpm-0.0.3-py3-none-any.whl size=20707 sha256=bb0b14fd69ddc8e698f86aede486b618b7d1cf44dbadcf3d80c66ca06a1a5bf9
  Stored in directory: C:\Users\sungwoo\AppData\Local\Temp\pip-ephem-wheel-cache-pa5a6mvq\wheels\dc\72\dd\d30f04632dad3b07fc5fd9017cf6af44bedce51258901ca564
Successfully built torchpm


In [21]:
from torchpm import *
from torchpm.data import CSVDataset
from torchpm.parameter import *
from torchpm import covariate
import torch as tc
import numpy as np
import matplotlib.pyplot as plt

# Model Building

In [22]:
dataset_file_path = 'https://raw.githubusercontent.com/yeoun9/torchpm/main/examples/THEO_cov_searching.csv'
dataset_np = np.loadtxt(dataset_file_path, delimiter=',', dtype=np.float32, skiprows=1)
device = tc.device("cuda:0" if tc.cuda.is_available() else "cpu")
column_names = ['ID', 'AMT', 'TIME', 'DV', 'CMT', "MDV", "RATE", 'BWT', 'fixed', 'rand-1+1', 'norm(0,1)', 'BWT-0.5+0.5']
dataset = CSVDataset(dataset_np, column_names, device)

class BasementModel(predfunction.PredictionFunctionByTime) :
    
    def _set_estimated_parameters(self):
        self.theta_0 = Theta(0., 1.5, 10.)
        self.theta_1 = Theta(0., 30., 100.)
        self.theta_2 = Theta(0, 0.08, 1)

        self.eta_0 = Eta()
        self.eta_1 = Eta()
        self.eta_2 = Eta()

        self.eps_0 = Eps()
        self.eps_1 = Eps()

        self.gut_model = linearode.Comp1GutModelFunction()
    
    def _calculate_parameters(self, covariates):
        covariates['k_a'] = self.theta_0()*tc.exp(self.eta_0())
        covariates['v'] = self.theta_1()*tc.exp(self.eta_1())#*para['BWT']/70
        covariates['k_e'] = self.theta_2()*tc.exp(self.eta_2())
        covariates['AMT'] = tc.tensor(320., device=self.dataset.device)

    def _calculate_preds(self, t, p):
        dose = p['AMT'][0]
        k_a = p['k_a']
        v = p['v']
        k_e = p['k_e']
        comps = self.gut_model(t, k_a, k_e, dose)
        return comps[1]/v
        
    def _calculate_error(self, y_pred, p):
        p['v_v'] = p['v'] 
        return y_pred +  y_pred * self.eps_0() + self.eps_1()

dependent_parameter_names = ['k_a', 'v', 'k_e']
dependent_parameter_initial_values = [[0,1.4901,2],[30,32.4667,34],[0,0.08,0.1]]
independent_parameter_names = ['BWT', 'fixed', 'rand-1+1', 'norm(0,1)', 'BWT-0.5+0.5']

searcher = covariate.DeepCovariateSearching(dataset=dataset,
                                BaseModel=BasementModel,
                                dependent_parameter_names=dependent_parameter_names,
                                independent_parameter_names=independent_parameter_names,
                                dependent_parameter_initial_values=dependent_parameter_initial_values,
                                eps_names=['eps_0', 'eps_1'],
                                omega = Omega([0.4397,
                                                0.0575,  0.0198, 
                                                -0.0069,  0.0116,  0.0205], False, requires_grads=True),
                                sigma = Sigma([[0.0177], [0.0762]], [True, True], requires_grads=[True, True]))
print(searcher.run())

NameError: name 'models' is not defined